In [2]:
import pymeshfix
import os
import datetime
import calcification_Module as cm
from meshparty import trimesh_io
import datajoint as dj
import numpy as np
import time

In [3]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [4]:
#make sure there is a temp file in the directory, if not then make one
#if temp folder doesn't exist then create it
if (os.path.isdir(os.getcwd() + "/temp_meshlab")) == False:
    os.mkdir("temp_meshlab")
    
    

In [5]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,vertices=[], triangles=[]):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    file_loc = pathlib.Path.cwd() / "temp"
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

In [19]:
key = dict(segmentation=2,segment_id=648518346341351731)

In [20]:

print("Starting on "+str(key["segment_id"]))
global_time = time.time()
#get the mesh with the error segments filtered away
start_time = time.time()

file_loc = pathlib.Path.cwd() / "temp_meshlab"
filename = "neuron_" + str(key["segment_id"])
path_and_filename = str(file_loc / filename)

#get the mesh of the neurite
new_key = dict(segmentation=key["segmentation"],
               segment_id=key["segment_id"])
mesh = (ta3p100.Mesh & new_key).fetch1()
print(f"Step 1: Retrieving Mesh: {time.time() - start_time}")

new_key["vertices"] = mesh["vertices"]
new_key["triangles"] = mesh["triangles"]
#print(key)
# print(key)
#print("Step 2: Remove all error semgents")
start_time = time.time()
#pass the vertices and faces to pymeshfix to become watertight
meshfix = pymeshfix.MeshFix(new_key["vertices"],new_key["triangles"])
meshfix.repair(verbose=False,joincomp=True,remove_smallest_components=False)
print(f"Step 2: Pymesh shrinkwrapping: {time.time() - start_time}")

#print("Step 2: Writing Off File")
start_time = time.time()
#write the new mesh to off file
path_and_filename,filename,file_loc = write_Whole_Neuron_Off_file(str(new_key["segment_id"]),meshfix.v,meshfix.f)
print(f"Step 3: Writing shrinkwrap off file: {time.time() - start_time}")


#send to be skeletonized
start_time = time.time()
return_value = cm.calcification(path_and_filename)
if return_value > 0:
    raise Exception('skeletonization for neuron ' + str(new_key["segment_id"]) + 
                    ' did not finish... exited with error code: ' + str(return_value))
print(f"Step 4: Generating Skeleton: {time.time() - start_time}")




Starting on 648518346341351731
Step 1: Retrieving Mesh: 0.03668522834777832
Step 2: Pymesh shrinkwrapping: 0.7589833736419678
Done writing OFF file
Step 3: Writing shrinkwrap off file: 0.19801974296569824
Step 4: Generating Skeleton: 0.8511683940887451


In [ ]:
ta3p100.Neurite